In [1]:
import langchain
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from tqdm.autonotebook import tqdm, trange
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange
from langchain.llms import HuggingFaceHub

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.prompts import PromptTemplate
import os
from getpass import getpass
import warnings
warnings.filterwarnings("ignore")

C:\Users\Evoortsolutions\AppData\Local\Temp\ipykernel_13588\2933666298.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
#add website data
URL = ["https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/",
       "https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/"]
 
#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(content)

In [4]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
 
import os
from getpass import getpass
 
# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token
 
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking,embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5},)
query = "What is Selenium?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel[1])

page_content='EDA also known as Exploratory Data Analysis is a technique that is used to analyze the data through visualization and manipulation. For this project let us visualize the data of famous companies such as Nvidia, Google, Apple, Facebook, and so on.' metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/', 'title': 'Stock Price Prediction Project using TensorFlow - GeeksforGeeks'}


In [ ]:
prompt_template = """
You are an intelligent assistant designed to provide answers based on the provided context. Follow these instructions:

***instructions***
1. Contextual Understanding:
   - Carefully read and understand the given context.
   - Use only the information provided in the context to generate your answer.
   
2. Answer Generation:
   - If the question is directly related to the context, provide a detailed and helpful answer.
   - If the information needed to answer the question is not mentioned in the context, respond with "I don't know."

Provided Context:
{context}

Question:
{question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
 
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.1,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })


# Define RetrievalQA chain
chain = RetrievalQA.from_chain_type(
    model, 
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt":PROMPT},
    chain_type="stuff"
)

response = chain({"query": query})

In [ ]:
print(response['result'])

You are a chatbot which helps people to retrive answer based on given context. If query was in context.
If query is not there in context then stright say "I don't know."
Choice QuizzesAptitude for PlacementsPuzzles for InterviewsLanguagesCC++JavaPythonR TutorialC#SQLScalaPerlGo LanguageWeb DevelopmentHTMLCSSJavaScriptTypeScriptReactJSReact JS TutorialNextJSNode.jsPHPAngularJSjQueryWeb Design100 Days of Web DevelopmentCS

EDA also known as Exploratory Data Analysis is a technique that is used to analyze the data through visualization and manipulation. For this project let us visualize the data of famous companies such as Nvidia, Google, Apple, Facebook, and so on.

QuestionsFlaskFlask TutorialPython Flask ProjectsFlask Interview QuestionsPostmanGithubWeb Scrapping in PythonData Structures & AlgorithmsDSA for BeginnersData StructuresArraysMatrixStringsLinked ListStackQueueTreeGeneric TreeBinary TreeBinary Search

TemplatesAptitude PreparationPuzzlesCompany-Wise PreparationCompaniesColleg